Función de ChatGPT

Funcion para cloud function

gcloud functions deploy nombre_de_la_funcion \
    --runtime python310 \
    --trigger-resource nombre_del_bucket \
    --trigger-event google.storage.object.finalize

In [2]:
from google.cloud import bigquery
from google.cloud import storage

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

def cargar_a_bigquery(project_id, dataset_id, table_id, gcs_bucket, gcs_object):
    # Inicializa el cliente de BigQuery y Cloud Storage
    bq_client = bigquery.Client(project=project_id)
    gcs_client = storage.Client(project=project_id)

    # Define la ubicación del archivo en Cloud Storage
    gcs_uri = f'gs://{gcs_bucket}/{gcs_object}'

    # Configura la referencia a la tabla de BigQuery
    table_ref = bq_client.dataset(dataset_id).table(table_id)

    # Configura las opciones de carga
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.json,  # Ajusta según el formato de tu archivo
        skip_leading_rows=0,  # Si hay encabezados en el archivo, ajusta esta opción
        autodetect=True,  # Deja que BigQuery detecte automáticamente el esquema
    )

    # Inicia el trabajo de carga
    load_job = bq_client.load_table_from_uri(
        gcs_uri, table_ref, job_config=job_config
    )

    # Espera a que el trabajo de carga se complete
    load_job.result()

    print(f'Carga completada en la tabla {project_id}.{dataset_id}.{table_id}')

if __name__ == "__main__":
    # Configura tus valores específicos
    project_id = 'tu-proyecto'
    dataset_id = 'tu-dataset'
    table_id = 'tu-tabla'
    gcs_bucket = 'tu-bucket'
    gcs_object = 'ruta/del/archivo.csv'  # Ajusta la ruta del archivo en Cloud Storage

    cargar_a_bigquery(project_id, dataset_id, table_id, gcs_bucket, gcs_object)


In [ ]:
def cargar_a_bigquery(event, context):
    """Esta función se ejecuta cuando se detecta un nuevo archivo en el bucket."""
    bucket_name = event['bucket']
    file_name = event['name']

    # Configurar el cliente de BigQuery y Cloud Storage
    bq_client = bigquery.Client()
    gcs_client = storage.Client()

    # Especificar el dataset y la tabla de destino en BigQuery
    dataset_id = 'tu_dataset'
    table_id = 'tu_tabla'

    # Crear el URI del archivo en GCS
    file_uri = f'gs://{bucket_name}/{file_name}'

    # Configurar la carga del archivo a BigQuery
    job_config = bigquery.LoadJobConfig(
        schema=[...],  # Especifica el esquema de tu tabla
        source_format=bigquery.SourceFormat.CSV,  # O el formato de tu archivo
    )

    # Iniciar la carga del archivo
    load_job = bq_client.load_table_from_uri(file_uri, dataset_id, table_id, job_config=job_config)
    load_job.result()  # Esperar a que se complete la carga

    print(f'Archivo {file_uri} cargado exitosamente a BigQuery.')

In [ ]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd

def cargar_a_bigquery(event, context):
    """Esta función se ejecuta cuando se detecta un nuevo archivo en el bucket."""
    bucket_name = event['bucket']
    file_name = event['name']

    # Configurar el cliente de BigQuery y Cloud Storage
    bq_client = bigquery.Client()
    gcs_client = storage.Client()

    # Especificar el dataset y la tabla de destino en BigQuery
    dataset_id = 'tu_dataset'
    table_id = 'tu_tabla'

    # Crear el URI del archivo en GCS
    file_uri = f'gs://{bucket_name}/{file_name}'

    # Configurar la carga del archivo a BigQuery
    job_config = bigquery.LoadJobConfig(
        schema=[...],  # Especifica el esquema de tu tabla
        source_format=bigquery.SourceFormat.CSV,  # O el formato de tu archivo
    )

    # Descargar el archivo de Cloud Storage y cargarlo en un DataFrame de pandas
    bucket = gcs_client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    data = blob.download_as_text()
    df = pd.read_csv(pd.compat.StringIO(data))

    # Realizar transformaciones en el DataFrame (por ejemplo, eliminar duplicados)
    df_sin_duplicados = df.drop_duplicates()

    # Guardar el DataFrame modificado en un nuevo archivo CSV en Cloud Storage
    nuevo_file_name = f'processed/{file_name}'  # Cambia el nombre según tus necesidades
    nuevo_blob = bucket.blob(nuevo_file_name)
    nuevo_blob.upload_from_string(df_sin_duplicados.to_csv(index=False), content_type='text/csv')

    print(f'Transformación completada. Archivo procesado guardado en {nuevo_blob}')

    # Actualizar el URI del archivo con la ruta al archivo procesado
    file_uri_procesado = f'gs://{bucket_name}/{nuevo_file_name}'

    # Iniciar la carga del archivo procesado a BigQuery
    load_job = bq_client.load_table_from_uri(file_uri_procesado, dataset_id, table_id, job_config=job_config)
    load_job.result()  # Esperar a que se complete la carga

    print(f'Archivo {file_uri_procesado} cargado exitosamente a BigQuery.')


In [ ]:
from google.cloud import bigquery
from google.cloud import storage

def cargar_a_bigquery(event, context):

    # Configurar el cliente de BigQuery y Cloud Storage
    bq_client = bigquery.Client()

    # Especificar el dataset y la tabla de destino en BigQuery
    dataset_id = 'tu_dataset'
    table_id = 'tu_tabla'

    # Crear el URI del archivo en GCS
    file_uri = f'gs://{bucket_name}/{file_name}'

    # Configurar la carga del archivo a BigQuery
    job_config = bigquery.LoadJobConfig(
        schema=[...],  # Especifica el esquema de tu tabla
        source_format=bigquery.SourceFormat.CSV,  # O el formato de tu archivo
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Sobrescribir la tabla existente
    )

    # Iniciar la carga del archivo a BigQuery
    load_job = bq_client.load_table_from_uri(file_uri, dataset_id, table_id, job_config=job_config)
    load_job.result()  # Esperar a que se complete la carga

    print(f'Archivo {file_uri} cargado exitosamente a BigQuery.')
